In [1]:
import pandas as pd
import time

In [2]:
# Get Benefit info, 1.88 sec
t = time.process_time()
ben8 = pd.read_csv('DE1_0_2008_Beneficiary_Summary_File_Sample_2.csv')
ben9 = pd.read_csv('DE1_0_2009_Beneficiary_Summary_File_Sample_2.csv')
ben10 = pd.read_csv('DE1_0_2010_Beneficiary_Summary_File_Sample_2.csv')
ben_all = pd.concat([ben8, ben9, ben10])
print(time.process_time() - t)

1.8837049999999997


In [3]:
ben_all.shape

(343858, 32)

In [4]:
# Load Claim Files, 112.72 sec
# Inpatient
t = time.process_time()
claims_IP = pd.read_csv('DE1_0_2008_to_2010_Inpatient_Claims_Sample_2.csv')
# Outpatient
claims_OP = pd.read_csv('DE1_0_2008_to_2010_Outpatient_Claims_Sample_2.csv')
# Merge Carrier
claims_Carrier_A = pd.read_csv('DE1_0_2008_to_2010_Carrier_Claims_Sample_2A.csv')
claims_Carrier_B = pd.read_csv('DE1_0_2008_to_2010_Carrier_Claims_Sample_2B.csv')
claims_Carrier = pd.concat([claims_Carrier_A, claims_Carrier_B])
print(time.process_time() - t)

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (21,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,10,11,46,47,48,49,50,136,137,138,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,9,10,11,45,46,47,48,49,50,135,136,137,138,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


112.721762


In [5]:
# Birth Date format
print ('The data type of the Date of Birth column (BENE_BIRTH_DT) is ' + str(type(ben_all['BENE_BIRTH_DT'][0])))
# All values are recorded as the first of the month in which the beneficiary was born. 
# For example 19740401

The data type of the Date of Birth column (BENE_BIRTH_DT) is <class 'pandas.core.series.Series'>


In [6]:
#Write beneficiary features of interest for corresponding patients into a single dataframe for preprocessing

working_df = pd.DataFrame(index = ben10.index)
beneficiary_features = ['DESYNPUF_ID','BENE_SEX_IDENT_CD', 'BENE_RACE_CD', 'BENE_BIRTH_DT', 'SP_STATE_CODE', 
                        'BENE_COUNTY_CD','SP_ALZHDMTA', 'SP_CHF', 'SP_CHRNKIDN', 'SP_CNCR', 'SP_COPD', 'SP_DIABETES', 'SP_DEPRESSN', 
                        'SP_ISCHMCHT', 'SP_OSTEOPRS', 'SP_RA_OA', 'SP_STRKETIA', 'MEDREIMB_OP', 'BENRES_OP', 
                        'PPPYMT_OP', 'BENE_DEATH_DT']    

In [7]:
#Select beneficiary data corresponding to the sample size
working_df = ben10

#Drop features that are not of interest
for col in working_df:
    if col not in beneficiary_features:
        working_df.drop(col, axis = 1, inplace = True)

In [8]:
#Drop rows and columns where all elements are NULL
working_df = working_df.dropna(axis=1, how='all')
working_df = working_df.dropna(axis=0, how='all')

In [9]:
#Add hospitalization date to the working dataframe
working_df = pd.merge(working_df, claims_IP[['DESYNPUF_ID','CLM_ADMSN_DT']], how='left', on='DESYNPUF_ID')

In [10]:
#target_date - start date of forecasting patients hospitalization 
target_date = 20100101

In [11]:
#Create target variable Y showing 1 if patient has been hospitalized in the target dataframe
working_df['Y'] = working_df.CLM_ADMSN_DT.apply(lambda x: 1 if x >= target_date else 0)

In [12]:
working_df['Y'].describe()

count    141363.000000
mean          0.095683
std           0.294157
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: Y, dtype: float64

In [13]:
#This takes a long time to run, 201.67 sec
#If hospitalized, first admission date
t = time.process_time()
dates = {}
for patient in working_df.DESYNPUF_ID[working_df.Y==1]:
    hosp_dates = working_df[working_df.DESYNPUF_ID==patient].CLM_ADMSN_DT
    min_date_2010 = min(hosp_dates[hosp_dates>=20100101])
    dates[patient] = min_date_2010
df_dates = pd.DataFrame(dates, index = dates.keys())
working_df['fst_admsn_dt'] = working_df['DESYNPUF_ID'].map(dates)
print(time.process_time() - t)

201.67442100000005


In [14]:
import datetime
from datetime import date
import dateutil
from datetime import datetime
from dateutil import parser
import numpy as np

def clean_df(df):
    
    '''Takes as input a DataFrame, creates a dummy variable for the death date (0 if still alive, 1 if not).
    Calculates the column TRUE_AGE, according to the death date if the dummy variable indicating death is 1, 
    or according to today's date if the dummy variable is 0. Recodes the column BENE_SEX_IDENT_CD (Sex) as 
    1: Female, 0: Male. '''
    
    days_a_year = 365.24
    
    ## Filling the missing values with 0, as explained above
    df['BENE_DEATH_DT'] = df['BENE_DEATH_DT'].fillna(0)
    
    ## Creating the column DEATH_DUMMY, 1 if the patient is dead, 0 otherwise
    df['DEATH_DUMMY'] = [int(int(df['BENE_DEATH_DT'][k])>0) for k in range(len(df['BENE_DEATH_DT']))]
    df['BENE_BIRTH_DT'] = [str(j) for j in df['BENE_BIRTH_DT']]
    
    df['BENE_DEATH_DT'] = [str(int("%2.f" % float(j))) for j in df['BENE_DEATH_DT']]
    
    bday_date = [dateutil.parser.parse(date) for date in df['BENE_BIRTH_DT']]
    
    death_date = []
    for d in range(len(df)):
        if df['DEATH_DUMMY'][d] == 0:
            death_date.append(dateutil.parser.parse(str(20100101)))
        elif df['DEATH_DUMMY'][d] == 1:
            death_date.append(dateutil.parser.parse(df['BENE_DEATH_DT'][d]))
    
    df['TRUE_AGE'] = [((death_date[j] - bday_date[j])/days_a_year).days for j in range(len(df['BENE_BIRTH_DT']))]
    
        ## Sex recoded to (0:Male, 1:Female) 
    df['BENE_SEX_IDENT_CD'] = [int(df['BENE_SEX_IDENT_CD'][i] == 2) for i in range(len(df['BENE_SEX_IDENT_CD']))]
    
    return df

In [15]:
#Takes long time to run, 19.81 sec
t = time.process_time()
clean_df = clean_df(working_df)
print(time.process_time() - t)

19.812880000000007


In [16]:
working_df.head(200)

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,SP_STATE_CODE,BENE_COUNTY_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,...,SP_RA_OA,SP_STRKETIA,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,CLM_ADMSN_DT,Y,fst_admsn_dt,DEATH_DUMMY,TRUE_AGE
0,00000B48BCF4AD29,19230901,0,1,5,10,260,2,1,2,...,2,2,600.0,30.0,0.0,20080707.0,0,NaN,0,86
1,00000B48BCF4AD29,19230901,0,1,5,10,260,2,1,2,...,2,2,600.0,30.0,0.0,20080724.0,0,NaN,0,86
2,00000B48BCF4AD29,19230901,0,1,5,10,260,2,1,2,...,2,2,600.0,30.0,0.0,20080805.0,0,NaN,0,86
3,0000525AB30E4DEF,19201001,0,1,1,31,300,2,2,2,...,2,2,240.0,190.0,0.0,20080525.0,0,NaN,0,89
4,0000525AB30E4DEF,19201001,0,1,1,31,300,2,2,2,...,2,2,240.0,190.0,0.0,20080918.0,0,NaN,0,89
5,00009C897C3D8372,19320101,0,0,1,7,70,2,1,2,...,2,2,210.0,160.0,0.0,20080329.0,0,NaN,0,78
6,00009C897C3D8372,19320101,0,0,1,7,70,2,1,2,...,2,2,210.0,160.0,0.0,20080402.0,0,NaN,0,78
7,00009C897C3D8372,19320101,0,0,1,7,70,2,1,2,...,2,2,210.0,160.0,0.0,20080415.0,0,NaN,0,78
8,00009C897C3D8372,19320101,0,0,1,7,70,2,1,2,...,2,2,210.0,160.0,0.0,20080514.0,0,NaN,0,78
9,00009C897C3D8372,19320101,0,0,1,7,70,2,1,2,...,2,2,210.0,160.0,0.0,20080921.0,0,NaN,0,78


In [17]:
#Takes long time to run
#Change "first_ad_date" from 20101023.0 to 2010-10-23
t = time.process_time()
clean_df['first_ad_date']=clean_df.fst_admsn_dt.astype(str).apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))
print(time.process_time() - t)

35.17691599999995


In [18]:
#Add a fake admission date for people who has no hospitalization in 2010: 3000-01-01
clean_df.first_ad_date.fillna(datetime(3000, 1, 1), inplace = True)

In [19]:
# The start training date for everyone is 2008-01-01
clean_df['start_train_dt']=datetime(2008,1,1)

In [20]:
import datetime
import calendar

def add_months(sourcedate, months):
    mmonth = sourcedate.month - 1 + months
    yyear = int(sourcedate.year + mmonth / 12)
    mmonth = mmonth % 12 + 1
    dday = min(sourcedate.day,calendar.monthrange(yyear,mmonth)[1])
    return datetime.date(yyear,mmonth,dday)

# Make a new variable to indicate the end of the training date, 0.34 sec
# If 'Y'=1(admission in 2010), 'end_train_dt' is 6 months before the first admission date in 2010;
# If 'Y'=0(admission in 2010), 'end_train_dt' is 2010-06-30;

t = time.process_time()
clean_df['end_train_dt']=clean_df.first_ad_date.apply(lambda x: add_months(x,-6))
print(time.process_time() - t)

0.3356759999999781


In [21]:
#Create Member months
import math

out = []

for row in range(clean_df.shape[0]):
    num=((clean_df.end_train_dt[row].year - clean_df.start_train_dt[row].year)*365 
    +(clean_df.end_train_dt[row].month - clean_df.start_train_dt[row].month)*30.4
    +(clean_df.end_train_dt[row].day - clean_df.start_train_dt[row].day))/30.4
    
    out.append(math.ceil(num))

In [22]:
# Add member month, 0.04 sec
t = time.process_time()
clean_df['member_month']= pd.Series(out)
print(time.process_time() - t)

0.04422499999998308


In [23]:
#Write pre-selected data into a csv for cleaning and feature selection
working_df.to_csv('preprocessed_data.csv')

In [28]:
import pickle
pickle.dump(clean_df, open( "clean_df.p", "wb" ) )
pickle.dump(working_df, open( "working_df.p", "wb" ) )

In [24]:
# Get outpatient payment by patient
outpatient_payment = pd.DataFrame(claims_OP.groupby('DESYNPUF_ID').sum()['CLM_PMT_AMT'])
outpatient_payment['DESYNPUF_ID'] = outpatient_payment.index
outpatient_payment.head()

,CLM_PMT_AMT,DESYNPUF_ID
DESYNPUF_ID,,
00000B48BCF4AD29,2700.0,00000B48BCF4AD29
0000525AB30E4DEF,5440.0,0000525AB30E4DEF
00009C897C3D8372,2560.0,00009C897C3D8372
0002EC0BCA99CACF,200.0,0002EC0BCA99CACF
0002FABBFD6D1C5C,1610.0,0002FABBFD6D1C5C


In [25]:
#OP counts
outpatient = pd.merge(outpatient, clean_df[['DESYNPUF_ID', 'end_train_dt', 'Y', 'TRUE_AGE']], how='left', on='DESYNPUF_ID')
outpatient = outpatient.dropna(0, subset = ['CLM_FROM_DT'])
outpatient['CLM_FROM_DT'] = [str(int(x)) for x in outpatient['CLM_FROM_DT']]
outpatient['CLM_FROM_DT'] = [dateutil.parser.parse(str(date)).date() for date in outpatient['CLM_FROM_DT']]
outpatient_df_1 = outpatient.dropna(subset=['end_train_dt'])
outpatient_df_1['end_train_dt'] = [dateutil.parser.parse(str(date)).date() for date in outpatient_df_1['end_train_dt']]
date_mask = [outpatient_df_1['CLM_FROM_DT'].iloc[i]<outpatient_df_1['end_train_dt'].iloc[i] for i in range(len(outpatient_df_1))]
df_training = outpatient_df_1[date_mask]
op_training_df = op_training_df.drop_duplicates(subset='CLM_ID')

op_training_df = pd.merge(op_training_df, outpatient_payment[['DESYNPUF_ID', 'CLM_PMT_AMT']], how='left', on='DESYNPUF_ID')

op_training_df['CLM_PMT_AMT_x'] = op_training_df['CLM_PMT_AMT_x'].fillna(0)
## Removing Duplicates

op_training_clm_unique = op_training_df.drop_duplicates(subset='CLM_ID')
op_training_clm_unique.head(100)

## Counting Unique Claim IDs for each patient
op_number_perid = pd.DataFrame(op_training_clm_unique.groupby('DESYNPUF_ID').count()['CLM_ID'])

op_number_perid['DESYNPUF_ID'] = op_number_perid.index

NameError: name 'outpatient' is not defined